<a href="https://colab.research.google.com/github/NARAYANAREDDY37/Keras-Tuner/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras-tuner

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=93e80735b37b2e763bfbf4cb96cc68971bf0b147756f35ce6c74f3cd21f3c80a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=53fe9c8dab0f7e3c5cbcaf8734bb2b7a17e01cd2d4a411239a01c0751cbecc1e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Real_Combine.csv to Real_Combine.csv
User uploaded file "Real_Combine.csv" with length 61870 bytes


In [6]:
df=pd.read_csv('Real_Combine.csv')
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [7]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [9]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

# Hyperparameters

How many number of hidden layers we should have?

How many number of neurons we should have in hidden layers?

Learning Rate


In [10]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [11]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [12]:
tuner.search_space_summary()

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 0s 17ms/step - loss: nan - mean_absolute_error: nan - val_loss: 70.3230 - val_mean_absolute_error: 70.3230
Epoch 2/5
24/24 [==============================] - 0s 6ms/step - loss: nan - mean_absolute_error: nan - val_loss: 68.0689 - val_mean_absolute_error: 68.0689
Epoch 3/5
24/24 [==============================] - 0s 6ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.1166 - val_mean_absolute_error: 63.1166
Epoch 4/5
24/24 [===================

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 1s 26ms/step - loss: nan - mean_absolute_error: nan - val_loss: 105.6286 - val_mean_absolute_error: 105.6286
Epoch 2/5
24/24 [==============================] - 0s 18ms/step - loss: nan - mean_absolute_error: nan - val_loss: 99.3054 - val_mean_absolute_error: 99.3054
Epoch 3/5
24/24 [==============================] - 0s 18ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.0995 - val_mean_absolute_error: 64.0995
Epoch 4/5
24/24 [===============

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 1s 42ms/step - loss: nan - mean_absolute_error: nan - val_loss: 65.4788 - val_mean_absolute_error: 65.4788
Epoch 2/5
24/24 [==============================] - 1s 32ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.6809 - val_mean_absolute_error: 64.6809
Epoch 3/5
24/24 [==============================] - 1s 31ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.7108 - val_mean_absolute_error: 63.7108
Epoch 4/5
24/24 [=================

...

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 1s 25ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.2627 - val_mean_absolute_error: 64.2627
Epoch 2/5
24/24 [==============================] - 0s 16ms/step - loss: nan - mean_absolute_error: nan - val_loss: 47.6770 - val_mean_absolute_error: 47.6770
Epoch 3/5
24/24 [==============================] - 0s 15ms/step - loss: nan - mean_absolute_error: nan - val_loss: 62.5925 - val_mean_absolute_error: 62.5925
Epoch 4/5
24/24 [=================

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 1s 27ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.0894 - val_mean_absolute_error: 64.0894
Epoch 2/5
24/24 [==============================] - 0s 18ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.7721 - val_mean_absolute_error: 64.7721
Epoch 3/5
24/24 [==============================] - 0s 19ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.6595 - val_mean_absolute_error: 63.6595
Epoch 4/5
24/24 [=================

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


INFO:tensorflow:Oracle triggered exit


In [15]:
tuner.results_summary()